In [1]:
import pandas as pd


# DataFrame

* DataFrame表示矩阵的数据表，包含已排序的列集合
    * 每一列的值可以是不同的类型。
    * DataFrame既有行索引，也有列索引 == 共享相同索引的Series的字典(存在一个字典， 字典中的每个键对应了一个Series对象， *所有Series对象的index都相同*)
    * DataFrame数据存储在一个二维块， 而不是列表，字典或其他一维数据的集合

* DataFrame属于二维数据，但是利用分层索引在DataFrame中展现更高维度的数据 —— 类似关系表的外键

* 可以将嵌套一维数据的字典作为参数传递

In [2]:
data = {
    'state': ["Ohio", "Ohio", "Ohio", "Nevada", "Nevada", "Nevada"],
    'year': [2000, 2001, 2002, 2003, 2001, 2002],
    'pop': [1.4, 2.3, 6.5, 7.4, 1., 4.3]
}

In [3]:
frame = pd.DataFrame(data)

In [4]:
frame

,state,year,pop
0,Ohio,2000,1.4
1,Ohio,2001,2.3
2,Ohio,2002,6.5
3,Nevada,2003,7.4
4,Nevada,2001,1.0
5,Nevada,2002,4.3


* head属性， 可以获取DataFrame的前5行数据
* tail属性， 可以获取DataFrame的后5行数据

In [5]:
frame.head()

,state,year,pop
0,Ohio,2000,1.4
1,Ohio,2001,2.3
2,Ohio,2002,6.5
3,Nevada,2003,7.4
4,Nevada,2001,1.0


In [6]:
frame.tail()

,state,year,pop
1,Ohio,2001,2.3
2,Ohio,2002,6.5
3,Nevada,2003,7.4
4,Nevada,2001,1.0
5,Nevada,2002,4.3


* 类似Series，可以传递一个索引列表， 对DataFrame的索引进行排序
    * columns， DataFrame的索引被定义为列(columns)

* DataFrame 也可以传递Series的索引，效果与Series相同

In [7]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'dept'], index=['one', 'two', 'three', 'four', 'five', 'six'])

In [8]:
frame2

,year,state,pop,dept
one,2000,Ohio,1.4,NaN
two,2001,Ohio,2.3,NaN
three,2002,Ohio,6.5,NaN
four,2003,Nevada,7.4,NaN
five,2001,Nevada,1.0,NaN
six,2002,Nevada,4.3,NaN


* 使用类似字典的索引方式，获取对应的Series对象


In [9]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

* 使用属性获取对应的Series

In [10]:
frame.year

0    2000
1    2001
2    2002
3    2003
4    2001
5    2002
Name: year, dtype: int64

***
DataFrame[columns] 对任意的列名有效
DataFrame.columns 只在列名有效
<br>
***

* loc 获取行

In [11]:
frame2.loc['three']

year     2002
state    Ohio
pop       6.5
dept      NaN
Name: three, dtype: object

* 利用pandas切片对视图进行修改，处理NaN

In [12]:
frame2.dept = 16.5

In [13]:
frame2

,year,state,pop,dept
one,2000,Ohio,1.4,16.5
two,2001,Ohio,2.3,16.5
three,2002,Ohio,6.5,16.5
four,2003,Nevada,7.4,16.5
five,2001,Nevada,1.0,16.5
six,2002,Nevada,4.3,16.5


* 可以用一个一维数据进行赋值，但是长度必须与DataFrame相同

In [14]:
frame2.dept = range(6)

In [15]:
frame2

,year,state,pop,dept
one,2000,Ohio,1.4,0
two,2001,Ohio,2.3,1
three,2002,Ohio,6.5,2
four,2003,Nevada,7.4,3
five,2001,Nevada,1.0,4
six,2002,Nevada,4.3,5


* 特别的， 将Series作为一列传递时，Series将会按照DataFrame的索引重新排列
* 需要注意，这个操作会对整列进行修改，如果DataFrame的索引不在Series中，则对应的值会被修改为NaN
* 如果Series的索引不在DataFrame中，则这一条目会被DataFame排除

In [16]:
val = pd.Series([-1.2, -1.4, -1.9, 3.3], index=['two', 'four', 'five', 'seven'])

In [17]:
frame2['dept'] = val

In [18]:
frame2

,year,state,pop,dept
one,2000,Ohio,1.4,NaN
two,2001,Ohio,2.3,-1.2
three,2002,Ohio,6.5,NaN
four,2003,Nevada,7.4,-1.4
five,2001,Nevada,1.0,-1.9
six,2002,Nevada,4.3,NaN


* 如果DataFrame索引的列不存在，这个方法会为DataFrame创建新的列
* 使用属性索引(DataFrame.columns)的方法无法创建
    * 程序可以正常运行，但是不会有结果

In [19]:
frame2['eastern'] = frame2.state == "Ohio"

In [20]:
frame2

,year,state,pop,dept,eastern
one,2000,Ohio,1.4,NaN,True
two,2001,Ohio,2.3,-1.2,True
three,2002,Ohio,6.5,NaN,True
four,2003,Nevada,7.4,-1.4,False
five,2001,Nevada,1.0,-1.9,False
six,2002,Nevada,4.3,NaN,False


* del 删除列 | 行

In [21]:
# 同样的删除列，也不能使用属性的方式索引要删除的列，且这样做会报错
del frame2['eastern']

In [22]:
frame2

,year,state,pop,dept
one,2000,Ohio,1.4,NaN
two,2001,Ohio,2.3,-1.2
three,2002,Ohio,6.5,NaN
four,2003,Nevada,7.4,-1.4
five,2001,Nevada,1.0,-1.9
six,2002,Nevada,4.3,NaN


* 使用嵌套字典创建DataFrame
* DataFrame会根据根据所有子字典的键的集合创建Series，并在各自对应的列下找到对应的索引进行填充
* 其余的值填充NaN

In [23]:
pop = {
    "Nevada": {
        '2001': 2.4,
        '2002': 2.0
    },
    "Ohio": {
        "2000": 1.5,
        "2001": 1.7,
        "2002": 3.5
    }
}

In [24]:
frame3 = pd.DataFrame(pop)

In [25]:
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.0,3.5
2000,NaN,1.5


* 使用类似numpy的方法 对DataFrame进行转置
* 转置操作不会修改视图

In [26]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.0,NaN
Ohio,1.7,3.5,1.5


In [27]:
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.0,3.5
2000,NaN,1.5


* 显式指明索引，定义索引的顺序， 索引不会自动排序
* 特别的如果index的类型是数据的话，DataFrame会自动进行递增排序

In [28]:
pd.DataFrame(pop, index=['2001', '2002', '2003'])

,Nevada,Ohio
2001,2.4,1.7
2002,2.0,3.5
2003,NaN,NaN


* 使用name属性对DataFrame的行和列进行命名
* DataFrame.index.name
* DataFrame.columns.name

In [29]:
frame3.index.name = 'year'

In [30]:
frame3.columns.name = 'state'

In [31]:
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.0,3.5
2000,NaN,1.5


* DataFrame的values会将包含在DataFrame中的数据以二维ndarray的形式返回

In [32]:
vals = frame3.values

In [33]:
vals

array([[2.4, 1.7],
       [2. , 3.5],
       [nan, 1.5]])

In [34]:
type(vals)

numpy.ndarray